In [19]:
import tenseal.sealapi as seal
from polynomial import enc_poly
import util
from numpy.polynomial.polynomial import polymul

In [20]:
parms = seal.EncryptionParameters(seal.SCHEME_TYPE.CKKS)
# How large should plaintexts be? This determines number of available levels for a certain bit size, number of data slots and security. 
poly_modulus_degree = 2**15
# Bit size for primes in modulus chain. Larger -> more available options, slower performance. 
bit_size = 50
# How many data levels do we need?
levels = 11
# Create modulus chain
mod_chain_bit_size = [bit_size for k in range(levels)]
# First and last primes are "special" in SEAL, and should always be 60 bit primes.  
mod_chain_bit_size.insert(0,60)
mod_chain_bit_size.append(60)

# Set parameters (this involves involves finding candidate primes for the modulus chain)
parms.set_poly_modulus_degree(poly_modulus_degree)
parms.set_coeff_modulus(seal.CoeffModulus.Create(poly_modulus_degree, mod_chain_bit_size))
scale = pow(2.0, bit_size)

# What are the specific primes in the modulus chain? These help calculate exact scales 
# of rescaled ciphertexts later on in this notebook. 
primes = [modulus.value() for modulus in parms.coeff_modulus()]

# The creation of a SEAL context does not include a check to see if parameter settings
# are valid. This is done later during key creation. 
context = seal.SEALContext(parms,True,seal.SEC_LEVEL_TYPE.TC128)

util.print_parameters(context)

print("|\t Max Bit Count: " + str(seal.CoeffModulus.MaxBitCount(poly_modulus_degree, seal.SEC_LEVEL_TYPE.TC128)))

/
|Encryption parameters: 
|	scheme: CKKS
|	poly_modulus_degree: 32768
|	coeff_modulus_size: 670 (60 50 50 50 50 50 50 50 50 50 50 50 60) bits
|	 Max Bit Count: 881


All keys geneated using `seal::KeyGenerator`. We also need an `encoder`, `evaluator`, `encryptor`, and `decryptor`. 

In [21]:

keygen = seal.KeyGenerator(context)
secret_key = keygen.secret_key()

public_key = seal.PublicKey()
keygen.create_public_key(public_key)

relin_keys = seal.RelinKeys()
keygen.create_relin_keys(relin_keys)

galois_keys = seal.GaloisKeys()
keygen.create_galois_keys()

encryptor = seal.Encryptor(context, public_key)
evaluator = seal.Evaluator(context)
decryptor = seal.Decryptor(context, secret_key)

encoder = seal.CKKSEncoder(context)
slot_count = seal.CKKSEncoder.slot_count(encoder)

print('Number of slots: ' + str(slot_count))

Number of slots: 16384


Let $x$ be the encrypted difference between two number in $a,b\in[0,1]$, such that $x\in[-1,1]$.

In [22]:
a=0.2;b=0.8

x = a-b
x_plain = seal.Plaintext()
encoder.encode(x,scale, x_plain)

x_enc = seal.Ciphertext()
encryptor.encrypt(x_plain, x_enc)

The polynomial we want to calculate is
* $g(x) = (35x-35x^3+21x^5-5x^7)/2^4$

which is done using the function `enc_poly` from the `polynomial` module. 

In [23]:
coeffs = [35, -35, 21, -5]
degrees = [1, 3, 5, 7]

power = 4

enc_g_x = enc_poly(x_enc, coeffs,degrees, power, evaluator, context, encoder,  parms, relin_keys, encryptor)

Note that the resulting ciphertext has a different scale, $\tilde{\Delta}$, compared to the bit-sizes of the modulus chain. This means we can not achieve scale stabilization. 
To compensate for this, manually change the scale of the resulting CT 
such that it matches the original scale, $\Delta$. 

- Note that `enc_g_x` is an encryption of $\tilde{\Delta}g(x)$ with scale $\tilde{\Delta}$
- Multiplying with plaintext $\Delta/\tilde{\Delta}$ which has scale $\tilde{\Delta}$ will yield an encryption of $\Delta g(x)$ with scale $\tilde{\Delta}^2$

Multiplying by the plain factor $\Delta/\tilde{\Delta}$ and manually changing the scale gives an encryption of $\Delta g(x)$ with the correct scale factor.

In [24]:
util.print_info(enc_g_x, decryptor, context, encoder, (35*x-35*x**3+21*x**5-5*x**7)/2**4)

# Code below introduces additional error. Question is if 
# this added error in practice eliminates the benefit
# of using the composite polynomial. 

scaling_factor = scale/enc_g_x.scale/2**power

plain_factor = seal.Plaintext()
encoder.encode(scaling_factor, enc_g_x.scale, plain_factor)
evaluator.mod_switch_to_inplace(plain_factor, enc_g_x.parms_id())

evaluator.multiply_plain_inplace(enc_g_x, plain_factor)
evaluator.rescale_to_next_inplace(enc_g_x)
enc_g_x.scale = scale

plain_g_x = (35*x-35*x**3+21*x**5-5*x**7)/2**4
util.print_info(enc_g_x, decryptor, context, encoder, (35*x-35*x**3+21*x**5-5*x**7)/2**4)

	Enc. result:	-0.9333119703141972
	Plain result:	-0.933312
	Scale:	1.8014399111364388e+16
	Ch. ind:	7
	Enc. result:	-0.93331200541354
	Plain result:	-0.933312
	Scale:	1125899906842624.0
	Ch. ind:	6


Now we are ready to calculate $f(g(x))$ where 
* $f(x) = (4589x-16577x^3+25614x^5-12860x^7)/2^{10}$

In [25]:
# Not working, unsure why scale is out of bound. With this method,
# we should achieve scale stabilization. Check scale stabilization 
# in square and multiply. 
coeffs = [4589, -16577, 25614, -12860]
power = 10

plain_f_g_x = (4589*plain_g_x-16577*plain_g_x**3+25614*plain_g_x**5-12860*plain_g_x**7)/2**10
enc_compare = enc_poly(enc_g_x, coeffs, degrees, power, evaluator, context, encoder, parms, relin_keys, encryptor)

In [26]:
util.print_info(enc_compare, decryptor, context, encoder, plain_f_g_x)

	Enc. result:	-0.9885385047226911
	Plain result:	-0.9885385398999578
	Scale:	1.152921615000919e+18
	Ch. ind:	2
